In [12]:
%%writefile m.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

_term='labor'
_article_id='12'

path = 'stop_words_en.txt'
with open(path) as f:
    stop_words = set(f.read().split())


for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
        text = text.lower()
    except ValueError as e:
        continue

    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    if _term in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Valid docs,%d" % 1
    if article_id != _article_id:
        continue
    # Your code for mapper here.
    for word in words:
        if word in stop_words:
            continue
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        if word == _term:
            print >> sys.stderr, "reporter:counter:Wiki stats,Target word,%d" % 1

Overwriting m.py


In [15]:
%%writefile p.py

import sys, re, math

def g():
    Nt, N, Dt = 0, 1, 1
    for line in sys.stdin:
        if "Target word=" in line:
            r=re.match("\s+{0}=(\d+)".format("Target word"), line)
            if r is not None:
                Nt=int(r.group(1))
        if "Total words=" in line:
            r=re.match("\s+{0}=(\d+)".format("Total words"), line)
            if r is not None:
                N=int(r.group(1))
        if "Valid docs=" in line:
            r=re.match("\s+{0}=(\d+)".format("Valid docs"), line)
            if r is not None:
                Dt=int(r.group(1))
    return (1/math.log(1. + Dt)) * Nt / N

if __name__ == '__main__':
    print(g())

Overwriting p.py


In [18]:
%%writefile r.py
import sys

current_key = None
word_sum = 0

_term='labor'

path = 'stop_words_en.txt'
with open(path) as f:
    stop_words = set(f.read().split())

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        if key != _term:
            continue
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)


Writing r.py


In [19]:
%%bash

hdfs dfs -rm -r -skipTrash tfidf_output > /dev/null

yarn jar \
/opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="TFIDF" \
    -D mapreduce.job.reduces=1 \
    -files m.py,r.py,/datasets/stop_words_en.txt \
    -mapper "python m.py" \
    -reducer "python r.py" \
    -input /data/wiki/en_articles_part \
    -output tfidf_output > /dev/null 2>stderr_logs.txt
    
cat stderr_logs.txt | python p.py


0.000350468962109862
